In [1]:
import pandas as pd
from ast import literal_eval

In [2]:
credits_path = 'data/credits.csv'
keywords_path = 'data/keywords.csv'
links_path = 'data/links_small.csv'
movies_path = 'data/movies_metadata.csv'
ratings_path = 'data/ratings_small.csv'

In [3]:
df = pd.read_csv(movies_path)
df = df[df['id'] != '1997-08-20']
df = df[df['id'] != '2012-09-29']
df = df[df['id'] != '2014-01-01']
df.drop_duplicates(subset=['id'], keep='first', inplace=True)

/var/folders/ll/x1qtww316xj02cjzd2j861sr0000gn/T/ipykernel_40033/2066012012.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(movies_path)


In [4]:
# Funciones auxiliares
def is_dict(element):
    return isinstance(element, dict)

def string_to_dic(dic_str):
    if not isinstance(dic_str, str):
        return dic_str
    return literal_eval(dic_str)

def get_dic_id(dic):
    if type(dic) != dict:
        return ""
    return int(dic["id"])

def get_unique_rows(cdf, column):
    cdf[column] = cdf[column].apply(string_to_dic)
    cdf = cdf.explode(column)
    cdf = cdf[pd.notna(cdf[column])]
    cdf = cdf[cdf[column].apply(lambda x: is_dict(x))]
    return cdf

def dic_to_new_df(df, col_name, columns, id='id'):
    # Extract 'id' and 'name' from the third column
    for c in columns:
        df[c] = df[col_name].apply(lambda x: x[c])

    # Create a new DataFrame with only 'id' and 'name'
    new_df = df[columns]
    new_df = new_df.drop_duplicates(subset=[id])
    new_df = new_df.reset_index(drop=True)

    return new_df

def nm_tables(df, column, id_name, extract_col):
    df = df[pd.notna(df[column])]
    df[column] = df[column].apply(string_to_dic)

    # Explode the 'countries' column to create separate rows for each country
    df = df.explode(column)
    df = df[pd.notna(df[column])]
    df = df[df[column].apply(lambda x: is_dict(x))]

    # Extract the 'iso_3166_1' values into a new column 'country_code'
    df[extract_col] = df[column].apply(lambda x: x[extract_col])

    # Select only the relevant columns
    df = df[[id_name, extract_col]]
    df = df.reset_index(drop=True)

    return df

def crew_cast_tables(df, column):
    # Separate the crew information
    df_crew = pd.json_normalize(df[column])

    # Reset indexes
    df = df.reset_index(drop=True)
    df_crew = df_crew.reset_index(drop=True)

    # Merge the DataFrames
    movie_merged = pd.concat([df, df_crew], axis=1)

    # Drop the original 'crew' column
    movie_merged.drop(column, axis=1, inplace=True)

    movie_merged = movie_merged.rename(columns={'id': 'person_id'})

    return movie_merged


In [5]:
# movie_basics
# -------------
# id
# title
# original_title
# release_date
# collection
# original_language
# adult
# status
# -------------

# List of column names you want to select
selected_columns = ['id', 'title', 'original_title', 'release_date', 'belongs_to_collection', 'original_language', 'adult', 'status']

# Create a new DataFrame with the selected columns
movie_basics = df[selected_columns].copy()
movie_basics['belongs_to_collection'] = movie_basics['belongs_to_collection'].apply(string_to_dic)

# Convertir collection
movie_basics['belongs_to_collection'] = movie_basics['belongs_to_collection'].apply(get_dic_id)

# Renombrar collection
movie_basics = movie_basics.rename(columns={'belongs_to_collection': 'collection'})

movie_basics = movie_basics[movie_basics['id'] != '1997-08-20']
movie_basics = movie_basics[movie_basics['id'] != '2012-09-29']
movie_basics = movie_basics[movie_basics['id'] != '2014-01-01']

movie_basics.drop_duplicates(subset=['id'], keep='first', inplace=True)
movie_basics.to_csv('proc_movie_basics.csv', index=False)
movie_basics.head(10)

,id,title,original_title,release_date,collection,original_language,adult,status
0,862,Toy Story,Toy Story,1995-10-30,10194,en,False,Released
1,8844,Jumanji,Jumanji,1995-12-15,,en,False,Released
2,15602,Grumpier Old Men,Grumpier Old Men,1995-12-22,119050,en,False,Released
3,31357,Waiting to Exhale,Waiting to Exhale,1995-12-22,,en,False,Released
4,11862,Father of the Bride Part II,Father of the Bride Part II,1995-02-10,96871,en,False,Released
5,949,Heat,Heat,1995-12-15,,en,False,Released
6,11860,Sabrina,Sabrina,1995-12-15,,en,False,Released
7,45325,Tom and Huck,Tom and Huck,1995-12-22,,en,False,Released
8,9091,Sudden Death,Sudden Death,1995-12-22,,en,False,Released
9,710,GoldenEye,GoldenEye,1995-11-16,645,en,False,Released


In [6]:
# movie_additional
# -------------
# id
# imdb_id
# homepage
# budget
# overview
# popularity
# vote_average
# vote_count
# poster_path
# tagline
# video
# revenue
# runtime
# -------------

# List of column names you want to select
selected_columns = ['id', 'imdb_id', 'homepage', 'budget', 'overview', 'popularity', 'vote_average', 'vote_count', 'poster_path', 'tagline', 'video', 'revenue', 'runtime']

# Create a new DataFrame with the selected columns
movie_additional = df[selected_columns].copy()

movie_additional['vote_count'].fillna(0, inplace=True)
movie_additional['vote_count'] = movie_additional['vote_count'].astype(int)
movie_additional['vote_count'].replace(0, 'null')

movie_additional['revenue'].fillna(0, inplace=True)
movie_additional['revenue'] = movie_additional['vote_count'].astype(int)
movie_additional['revenue'].replace(0, 'null')

# Convertir collection
movie_additional.to_csv('proc_movie_additional.csv', index=False)

In [7]:
# genres
# -------------
# genre_id
# name
# -------------
# Convertir genres
genres = df[['id', 'genres']].copy()
unique_genres_df = get_unique_rows(genres, "genres")
unique_genres_df = dic_to_new_df(unique_genres_df, 'genres', ['id', 'name'])

# Renombrar id a genre_id
unique_genres_df = unique_genres_df.rename(columns={'id': 'genre_id'})

# Imprimir dataframe final (primeros 10 datos)
unique_genres_df.to_csv('proc_genres.csv', index=False)

In [8]:
# movie_genres
# -------------
# movie_id
# genre_id
# -------------
movie_genres = df[['id', 'genres']]
movie_genres = movie_genres.rename(columns={'id': 'movie_id'})
movie_genres = nm_tables(movie_genres, "genres", "movie_id", "id")
movie_genres = movie_genres.rename(columns={'id': 'genre_id'})
movie_genres.to_csv('proc_movie_genres.csv', index=False)

In [9]:
# production_companies
# -------------
# company_id
# name
# -------------
# Convertir companies
companies = df[['id', 'production_companies']].copy()
unique_companies = get_unique_rows(companies, 'production_companies')
unique_companies = dic_to_new_df(unique_companies, 'production_companies', ['id', 'name'])
unique_companies = unique_companies.rename(columns={'id': 'company_id'})
unique_companies.to_csv('proc_production_companies.csv', index=False)

In [10]:
# movie_production_companies
# -------------
# movie_id
# company_id
# -------------
movie_production_companies = df[['id', 'production_companies']]
movie_production_companies = movie_production_companies.rename(columns={'id': 'movie_id'})
movie_production_companies = nm_tables(movie_production_companies, "production_companies", "movie_id", "id")
movie_production_companies = movie_production_companies.rename(columns={'id': 'company_id'})
movie_production_companies.to_csv('proc_movie_production_companies.csv', index=False)

/var/folders/ll/x1qtww316xj02cjzd2j861sr0000gn/T/ipykernel_40033/133971777.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(string_to_dic)


In [11]:
# languages
# -------------
# iso_639_1
# name
# -------------
# Convertir languages
languages = df[['id', 'spoken_languages']].copy()
unique_languages = get_unique_rows(languages, 'spoken_languages')
unique_languages = dic_to_new_df(unique_languages, 'spoken_languages', ['iso_639_1', 'name'], 'iso_639_1')

extra_languages = df[['original_language']]
extra_languages.head(10)
extra_languages = extra_languages.rename(columns={'original_language': 'iso_639_1'})

languages = pd.concat([unique_languages, extra_languages], ignore_index=True).drop_duplicates(subset='iso_639_1', keep='first')

unique_languages.to_csv('proc_languages.csv', index=False)

In [12]:
unique_languages.head(10)

,iso_639_1,name
0,en,English
1,fr,Français
2,es,Español
3,de,Deutsch
4,ru,Pусский
5,la,Latin
6,nl,Nederlands
7,cn,广州话 / 廣州話
8,zh,普通话
9,hu,Magyar


In [13]:
# movie_languages
# -------------
# movie_id
# language_code
# -------------
movie_languages = df[['id', 'spoken_languages']]
movie_languages = movie_languages.rename(columns={'id': 'movie_id'})
movie_languages = nm_tables(movie_languages, "spoken_languages", "movie_id", "iso_639_1")
movie_languages = movie_languages.rename(columns={'iso_639_1': 'language_code'})
movie_languages.to_csv('proc_movie_languages.csv', index=False)

/var/folders/ll/x1qtww316xj02cjzd2j861sr0000gn/T/ipykernel_40033/133971777.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(string_to_dic)


In [14]:
# keywords
# -------------
# keyword_id
# name
# -------------
# Convertir keywords
keywords = pd.read_csv(keywords_path)
keywords.drop_duplicates(subset=['id'], keep='first', inplace=True)
unique_keywords = get_unique_rows(keywords, 'keywords')
unique_keywords = dic_to_new_df(unique_keywords, 'keywords', ['id', 'name'])
unique_keywords = unique_keywords.rename(columns={'id': 'keyword_id'})
unique_keywords.to_csv('proc_keywords.csv', index=False)

In [15]:
# movie_keywords
# -------------
# movie_id
# keyword_id
# -------------
movie_keywords = keywords[['id', 'keywords']]
movie_keywords = movie_keywords.rename(columns={'id': 'movie_id'})
movie_keywords = nm_tables(movie_keywords, "keywords", "movie_id", "id")
movie_keywords = movie_keywords.rename(columns={'id': 'keyword_id'})
movie_keywords.to_csv('proc_movie_keywords.csv', index=False)

In [16]:
# collections
# -------------
# id
# name
# poster_path
# backdrop_path
# -------------
# Convertir collections
collections = df[["id", "belongs_to_collection"]]
collections["dics"] = collections["belongs_to_collection"].apply(string_to_dic)
collections = collections[pd.notna(collections["belongs_to_collection"])]
collections = collections[collections["dics"].apply(lambda x: is_dict(x))]
unique_collections = dic_to_new_df(collections, 'dics', ['id', 'name', 'poster_path', 'backdrop_path'])
unique_collections.to_csv('proc_collections.csv', index=False)

/var/folders/ll/x1qtww316xj02cjzd2j861sr0000gn/T/ipykernel_40033/3128252940.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collections["dics"] = collections["belongs_to_collection"].apply(string_to_dic)


In [17]:
# countries
# -------------
# iso_3166_1
# name
# -------------
countries = df[['id', 'production_countries']].copy()
countries = get_unique_rows(countries, 'production_countries')
countries = dic_to_new_df(countries, 'production_countries', ['iso_3166_1', 'name'], 'iso_3166_1')
countries.to_csv('proc_countries.csv', index=False)

In [18]:
# movie_production_countries
# -------------
# movie_id
# country_code
# -------------
movie_production_countries = df[['id', 'production_countries']]
movie_production_countries = movie_production_countries.rename(columns={'id': 'movie_id'})
movie_production_countries = nm_tables(movie_production_countries, "production_countries", "movie_id", "iso_3166_1")
movie_production_countries = movie_production_countries.rename(columns={'iso_3166_1': 'country_code'})
movie_production_countries.to_csv('proc_movie_production_countries.csv', index=False)

/var/folders/ll/x1qtww316xj02cjzd2j861sr0000gn/T/ipykernel_40033/133971777.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].apply(string_to_dic)


In [19]:
# people
# -------------
# id
# name
# gender
# profile_path
# -------------
credits = pd.read_csv('data/credits.csv')
people1 = credits[['id', 'cast']].copy()
people1 = get_unique_rows(people1, 'cast')
people1 = dic_to_new_df(people1, 'cast', ['id', 'name', 'gender', 'profile_path'], 'id')

people2 = credits[['id', 'crew']].copy()
people2 = get_unique_rows(people2, 'crew')
people2 = dic_to_new_df(people2, 'crew', ['id', 'name', 'gender', 'profile_path'], 'id')

# Concatenate DataFrames row-wise
people = pd.concat([people1, people2], axis=0)

people.drop_duplicates(subset=['id'], keep='first', inplace=True)

# Reset the index
people.reset_index(drop=True, inplace=True)

people.to_csv('proc_people.csv', index=False)

In [20]:
# movie_crew
# -------------
# credit_id
# movie_id
# person_id
# department
# job
# profile_path
# -------------
movie_crew = credits[['id', 'crew']].copy()
movie_crew = movie_crew.rename(columns={'id': 'movie_id'})
movie_crew = get_unique_rows(movie_crew, 'crew')

movie_crew_merged = crew_cast_tables(movie_crew, 'crew')

movie_crew_merged = movie_crew_merged[['credit_id', 'movie_id', 'person_id', 'department', 'job', 'profile_path']]

movie_crew_merged.drop_duplicates(subset=['credit_id'], keep='first', inplace=True)

movie_crew_merged.to_csv('proc_movie_crew.csv', index=False)

In [21]:
# movie_cast
# -------------
# credit_id
# movie_id
# person_id
# character
# cast_id
# order
# -------------
movie_cast = credits[['id', 'cast']].copy()
movie_cast = movie_cast.rename(columns={'id': 'movie_id'})
movie_cast = get_unique_rows(movie_cast, 'cast')

movie_cast_merged = crew_cast_tables(movie_cast, 'cast')

movie_cast_merged = movie_cast_merged[['credit_id', 'movie_id', 'person_id', 'character', 'cast_id', 'order']]

movie_cast_merged.drop_duplicates(subset=['credit_id'], keep='first', inplace=True)

movie_cast_merged.to_csv('proc_movie_cast.csv', index=False)

In [22]:
# ratings
# -------------
# user_id
# movie_id
# rating
# timestamp
# -------------
ratings = pd.read_csv(ratings_path)
ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')  # 's' represents seconds since the Unix epoch
ratings.to_csv('proc_ratings.csv', index=False)

In [23]:
# link_ids
# -------------
# movie_id
# imdb_id
# tmpdb_id
# -------------
links = pd.read_csv(links_path)
links['tmdbId'].fillna(0, inplace=True)
links['tmdbId'] = links['tmdbId'].astype(int)
links['tmdbId'].replace(0, 'null')
links.to_csv('proc_link_ids.csv', index=False)